In [ ]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, root
import scipy.integrate

from ipywidgets import interact, FloatSlider

from bokeh.io import push_notebook, output_notebook
from bokeh.plotting import show, figure
from bokeh.layouts import column, row
from bokeh.models import PrintfTickFormatter
output_notebook(hide_banner=True)

# Course 4 : Explosion model for high order methods


## One equation case

$$
d_{\tau} \widetilde{\theta} = f(\widetilde{\theta}) = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \Bigg(1 - \frac{\widetilde{\theta}}{\overline T_r} \Bigg)
$$

In [ ]:
class explosion_with_consumption_1eq_model:

    def __init__(self, Tr, beta):
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * (1-theta/Tr) 
        return theta_dot
    
    def jac(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        exp = np.exp(theta/(1+(theta/beta)))
        return (exp/(1+theta/beta)**2) * (1-theta/Tr) - exp/Tr

In [ ]:
 def show_f_and_df():

    Tr = 200.0
    beta = 10.

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac
    
    theta = np.linspace(0, Tr, 1000)
    
    fig = figure(x_range=(0, Tr), plot_width=800, plot_height=300, title="Non-linéarité du terme source")
    fig.xaxis.axis_label = "\u03B8"
    fig.yaxis.axis_label = "Terme source : f(\u03B8)"
    fig.yaxis[0].formatter = PrintfTickFormatter(format="%6.2e")
    plt = fig.line(theta, fcn(0, theta), line_width=2)
    
    fig_df = figure(x_range=(0, Tr), plot_width=800, plot_height=300, title="Dérivée du terme source")
    fig_df.xaxis.axis_label = "\u03B8"
    fig_df.yaxis.axis_label = "Dérivée : df/d\u03B8"
    fig_df.yaxis[0].formatter = PrintfTickFormatter(format="%6.2e")
    plt_df = fig_df.line(theta, jac(0, theta), line_width=2)

    show(column(fig, fig_df), notebook_handle=True)
    
    @interact(beta=FloatSlider(min=5, max=100, step=5, value=10))
    def update(beta):
        emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
        fcn = emwc1eq.fcn        
        jac = emwc1eq.jac        
        plt.data_source.data = dict(x=theta, y=fcn(0, theta))
        plt_df.data_source.data = dict(x=theta, y=jac(0, theta))
        push_notebook()
    
show_f_and_df()

## Numerical solution

### One order method

In [ ]:
class ode_result:
    def __init__(self, y, t, rho=0):
        self.y = y
        self.t = t
        self.rho = rho

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [ ]:
def test_one_order_methods():
    
    Tr = 200.0
    beta = 10.

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    tini = 0.0
    tend = 5.0
    
    # Exemple d'utilisation : Prendre 500 pas de temps avec Beta=10, puis 400, 300, 200 et finalement 150
    nt = 201
    dt = (tend-tini) / nt

    print(f"  dt = {dt}")

    yini = (0.,)
    
    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    if (beta<11.1 and nt > 150): sol_fe = forward_euler(tini, tend, nt, yini, fcn)
    sol_be = backward_euler(tini, tend, nt, yini, fcn)

    fig_theta = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="\u03B8(t)")
    fig_theta.line(sol_ref.t, sol_ref.y[0], line_width=2, color="crimson", legend_label="Reference")
    if (beta<11.1 and nt > 150): fig_theta.line(sol_fe.t, sol_fe.y[0], line_width=2, color="green", legend_label="Euler explicite")
    fig_theta.line(sol_be.t, sol_be.y[0], line_width=2, legend_label="Euler implicite")
    fig_theta.legend.click_policy="hide"
    fig_theta.legend.location = "top_left"

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size): eig_vals_ref[it] = jac(0, sol_ref.y[:,it])

    if (beta<11.1 and nt > 150):
        eig_vals_fe = np.zeros(sol_fe.t.size)
        for it in range(sol_be.t.size): eig_vals_fe[it] = jac(0, sol_fe.y[:,it])

    eig_vals_be = np.zeros(sol_be.t.size)
    for it in range(sol_be.t.size): eig_vals_be[it] = jac(0, sol_be.y[:,it])


    fig_eig = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="Valeur propre")
    fig_eig.line(sol_ref.t, eig_vals_ref, color="crimson", line_width=2, legend_label="Reference")
    if (beta<11.1 and nt > 150): fig_eig.line(sol_fe.t, eig_vals_fe, color="green", line_width=2, legend_label="Euler explicite")
    fig_eig.line(sol_be.t, eig_vals_be, line_width=2, legend_label="Euler implicite")
    fig_eig.legend.click_policy="hide"

    fig_eigdt = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="Valeur propre * dt")
    if (beta<11.1 and nt > 150): fig_eigdt.line(sol_fe.t, eig_vals_fe*dt, color="green", line_width=2, legend_label="Euler explicite")
    fig_eigdt.line(sol_be.t, eig_vals_be*dt, line_width=2, legend_label="Euler implicite")
    fig_eigdt.legend.click_policy="hide"
    
    show(fig_theta)
    show(fig_eig)
    show(fig_eigdt)
    
test_one_order_methods()

### High order methods

In [ ]:
def rk3(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / nt
    t = np.linspace(tini, tend, nt+1)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt+1), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = np.array(fcn(tn, yn))
        k2 = np.array(fcn(tn + 0.5*dt, yn + dt*(0.5*k1)))
        k3 = np.array(fcn(tn + dt, yn + dt*(-k1 + 2*k2)))
        y[:,it+1] = yn + (dt/6)*(k1+4*k2+k3)

    return ode_result(y, t)

def rk4(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / nt
    t = np.linspace(tini, tend, nt+1)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt+1), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = np.array(fcn(tn, yn))
        k2 = np.array(fcn(tn + 0.5*dt, yn + dt*(0.5*k1)))
        k3 = np.array(fcn(tn + 0.5*dt, yn + dt*(0.5*k2)))
        k4 = np.array(fcn(tn + dt, yn + dt*k3))
        y[:,it+1] = yn + (dt/6)*(k1+2*k2+2*k3+k4)

    return ode_result(y, t)

def radauIIA_3(tini, tend, nt, yini, fcn):
    
    c1 = 1/3
    c2 = 1.
    
    a11 = 5/12
    a12 = -1/12
    a21 = 3/4
    a22 = 1/4
    
    b1 = 3/4
    b2 = 1/4
    
    a = np.array([[a11, a12], [a21, a22]])
    rho = np.max(np.abs(np.linalg.eigvals(np.abs(a))))

    dt = (tend-tini) / nt
    t = np.linspace(tini, tend, nt+1)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt+1), order='F')
    y[:,0] = yini_array
    
    #k0 = np.zeros(2*neq)
    k0 = np.concatenate((fcn(tini,yini_array), fcn(tini,yini_array)))   
        
    def g(k, *args):        
        yn, tn = args
        k1, k2 = np.split(k,2)         
        fk1 = fcn(tn+c1*dt, yn+dt*(a11*k1+a12*k2))
        fk2 = fcn(tn+c2*dt, yn+dt*(a21*k1+a22*k2))
        return k - np.concatenate((fk1, fk2))     
    
    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k = fsolve(g, k0, (yn, tn))
        k0 = np.copy(k)
        k1, k2 = np.split(k,2)
        y[:,it+1] = yn + dt*(b1*k1+b2*k2)
        
    return ode_result(y, t, rho=rho)

def radauIIA_5(tini, tend, nt, yini, fcn):
    
    r6 = 6**0.5
    
    c1 = 2/5 - r6/10
    c2 = 2/5 + r6/10
    c3 = 1.
    
    a11 = 11/45 - 7*r6/360
    a12 = 37/225 - 169*r6/1800
    a13 = -2/225 + r6/75
    
    a21 = 37/225 + 169*r6/1800
    a22 = 11/45 + 7*r6/360
    a23 = -2/225 - r6/75
    
    a31 = 4/9 - r6/36
    a32 = 4/9 + r6/36
    a33 = 1/9

    b1 = 4/9 - r6/36
    b2 = 4/9 + r6/36
    b3 = 1/9
    
    a = np.array([[a11, a12, a13], [a21, a22, a23], [a31, a32, a33]])
    rho = np.max(np.abs(np.linalg.eigvals(np.abs(a))))
    
    dt = (tend-tini) / nt
    t = np.linspace(tini, tend, nt+1)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt+1), order='F')
    y[:,0] = yini_array
    
    #k0 = np.zeros(3*neq)
    k0 = np.concatenate((fcn(tini,yini_array), fcn(tini,yini_array), fcn(tini,yini_array)))   

    def g(k, *args):        
        yn, tn = args
        k1, k2, k3 = np.split(k,3)         
        fk1 = fcn(tn+c1*dt, yn+dt*(a11*k1+a12*k2+a13*k3))
        fk2 = fcn(tn+c2*dt, yn+dt*(a21*k1+a22*k2+a23*k3))
        fk3 = fcn(tn+c3*dt, yn+dt*(a31*k1+a32*k2+a33*k3))
        return k - np.concatenate((fk1, fk2, fk3))     
    
    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k = fsolve(g, k0, (yn, tn))
        k0 = np.copy(k)
        k1, k2, k3 = np.split(k,3)
        y[:,it+1] = yn + dt*(b1*k1+b2*k2+b3*k3)
        
    return ode_result(y, t, rho=rho)

In [ ]:
def test_higher_order_methods():
    
    Tr = 200.0
    beta = 10.

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    tini = 0.0
    tend = 5.0
    
    # Exemple d'utilisation : Prendre 200 pas de temps avec Beta=10, puis 100, 90 et finalement 80
    nt = 250
    dt = (tend-tini) / (nt-1)

    print(f"  dt = {(tend-tini) / (nt-1)}")

    yini = (0.,)
    
    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    if (beta<21.1 and nt>129): sol_rk3 = rk3(tini, tend, nt, yini, fcn)
    if (beta<21.1): sol_rk4 = rk4(tini, tend, nt, yini, fcn)
    sol_radau3 = radauIIA_3(tini, tend, nt, yini, fcn)
    sol_radau5 = radauIIA_5(tini, tend, nt, yini, fcn)

    fig_theta = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="\u03B8(t)")
    fig_theta.line(sol_ref.t, sol_ref.y[0], line_width=2, color="crimson", legend_label="Reference")
    if (beta<21.1 and nt>129): fig_theta.line(sol_rk3.t, sol_rk3.y[0], line_width=2, color="green", legend_label="RK3")
    if (beta<21.1): fig_theta.line(sol_rk4.t, sol_rk4.y[0], line_width=2, color="cornflowerblue", legend_label="RK4")
    fig_theta.line(sol_radau3.t, sol_radau3.y[0], line_width=2, color="purple", legend_label="RadauIIA-3")
    fig_theta.line(sol_radau5.t, sol_radau5.y[0], line_width=2, color="goldenrod", legend_label="RadauIIA-5")
    fig_theta.legend.click_policy="hide"
    fig_theta.legend.location = "top_left"

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size): eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_radau3 = np.zeros(sol_radau3.t.size)
    for it in range(sol_radau3.t.size): eig_vals_radau3[it] = jac(0, sol_radau3.y[:,it])
    eig_vals_radau5 = np.zeros(sol_radau5.t.size)
    for it in range(sol_radau5.t.size): eig_vals_radau5[it] = jac(0, sol_radau5.y[:,it])
    if (beta<21.1  and nt>129):
        eig_vals_rk3 = np.zeros(sol_rk3.t.size)
        for it in range(sol_rk3.t.size): eig_vals_rk3[it] = jac(0, sol_rk3.y[:,it])
    if (beta<21.1):
        eig_vals_rk4 = np.zeros(sol_rk4.t.size)
        for it in range(sol_rk4.t.size): eig_vals_rk4[it] = jac(0, sol_rk4.y[:,it])

    fig_eig = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="Valeur propre")
    fig_eig.line(sol_ref.t, eig_vals_ref, color="crimson", line_width=2, legend_label="Reference")
    if (beta<21.1 and nt>129): fig_eig.line(sol_rk3.t, eig_vals_rk3, color="green", line_width=2, legend_label="RK3")
    if (beta<21.1): fig_eig.line(sol_rk4.t, eig_vals_rk4, color="cornflowerblue", line_width=2, legend_label="RK4")
    fig_eig.line(sol_radau3.t, eig_vals_radau3, line_width=2, color="purple", legend_label="RadauIIA-3")
    fig_eig.line(sol_radau5.t, eig_vals_radau5, line_width=2, color="goldenrod", legend_label="RadauIIA-5")
    fig_eig.legend.click_policy="hide"

    rho3 = sol_radau3.rho
    rho5 = sol_radau5.rho

    fig_eigdt = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="Valeur propre * dt * rho(A)")
    if (beta<21.1 and nt>129): fig_eigdt.line(sol_rk3.t, eig_vals_rk3*dt, color="green", line_width=2, legend_label="RK3")
    if (beta<21.1): fig_eigdt.line(sol_rk4.t, eig_vals_rk4*dt, color="cornflowerblue", line_width=2, legend_label="RK4")
    fig_eigdt.line(sol_radau3.t, eig_vals_radau3*dt*rho3, line_width=2, color="purple", legend_label="RadauIIA-3")
    fig_eigdt.line(sol_radau5.t, eig_vals_radau5*dt*rho5, line_width=2, color="goldenrod", legend_label="RadauIIA-5")
    fig_eigdt.legend.click_policy="hide"
    
    show(fig_theta)
    show(fig_eig)
    show(fig_eigdt)
    
test_higher_order_methods()

### SDIRK method

In [ ]:
########################################################################################
def DIRK_integration(fun, y0, t_span, nt, jacfun=None, bPrint=True):
    """ Performs the integration of the system dy/dt = f(t,y)
        from t=t_span[0] to t_span[1], with initial condition y(t_span[0])=y0.
        - fun      :  (function handle) model function (time derivative of y)
        - y0     :  (1D-array)        initial condition
        - t_span :  (1D-array)        array of 2 values (start and end times)
        - nt     :  (integer)         number of time steps
        - jacfun  :  (function handle, optional) function returning a 2D-array (Jacobian df/dy)
        """
    assert y0.ndim==1, 'y0 must be 0D or 1D'

    A = np.zeros((7,7))
    b = np.zeros(7)
    c = np.zeros(7)

    
    # Butcher table
    A[1,0] = 0.26
    A[1,1] = 0.26
    A[2,0] = 0.13
    A[2,1] = 0.84033320996790809
    A[2,2] = 0.26
    A[3,0] = 0.22371961478320505
    A[3,1] = 0.47675532319799699
    A[3,2] = -0.06470895363112615
    A[3,3] = 0.26
    A[4,0] = 0.16648564323248321
    A[4,1] = 0.10450018841591720
    A[4,2] = 0.03631482272098715
    A[4,3] = -0.13090704451073998
    A[4,4] = 0.26
    A[5,0] = 0.13855640231268224
    A[5,2] = -0.04245337201752043
    A[5,3] = 0.02446657898003141
    A[5,4] = 0.61943039072480676
    A[5,5] = 0.26
    A[6,0] = 0.13659751177640291
    A[6,2] = -0.05496908796538376
    A[6,3] = -0.04118626728321046
    A[6,4] = 0.62993304899016403
    A[6,5] = 0.06962479448202728
    A[6,6] = 0.26

    b[0] = 0.13659751177640291
    b[2] = -0.05496908796538376
    b[3] = -0.04118626728321046
    b[4] = 0.62993304899016403
    b[5] = 0.06962479448202728
    b[6] = 0.26

    c[1] = 0.52
    c[2] = 1.230333209967908
    c[3] = 0.895765984350076
    c[4] = 0.436393609858648
    c[5] = 1.0
    c[6] = 1.0

    out = scipy.integrate._ivp.ivp.OdeResult()
    t = np.linspace(t_span[0], t_span[1], nt+1)
    
    nx = np.size(y0)
    dt = (t_span[1]-t_span[0]) / nt
    s = np.size(b)

    y = np.zeros((y0.size, nt+1))
    y[:,0] = y0
    
    K= np.zeros((np.size(y0), s))
    unm1 = np.copy(y0)
    for it, tn in enumerate(t[:-1]):
        if bPrint:
            if np.mod(it,np.floor(nt/10))==0:
                print('\n{:.1f} %'.format(100*it/nt), end='')
            if np.mod(it,np.floor(nt/100))==0:
                print('.', end='')
        for isub in range(s): # go through each substep
            temp = np.zeros(np.shape(y0))
            for j in range(isub):
                temp    = temp  +  A[isub,j] * K[:,j]
            vi = unm1 + dt*( temp )

            if A[isub,isub]==0: # explicit step
                kni = fun(tn+c[isub]*dt, vi)
            else:
                 # solve the complete non-linear system via a Newton method
                tempfun = lambda kni: kni - fun(tn+c[isub]*dt, vi + dt*A[isub,isub]*kni)
                if 1:
                    kni = scipy.optimize.fsolve(func= tempfun,
                                        x0=K[:,0],
                                        fprime=None,
                                        # band=(5,5), #gradFun
                                        # epsfcn = 1e-7,
                                        args=(),)
                else:
                    kni = scipy.optimize.newton(func= tempfun,
                                        x0=K[:,0],
                                        fprime=None,
                                        # band=(5,5), #gradFun
                                        # epsfcn = 1e-7,
                                        args=(),)
            K[:,isub] = kni #fun(tn+c[isub]*dt, ui[:,isub])
        ## END OF QUADRATURE STEPS --> reaffect unm1
        for j in range(s):
            unm1[:] = unm1[:] + dt*b[j]*K[:,j]
        K[:,:] = 0.
        y[:,it+1] = unm1[:]
    # END OF INTEGRATION
    out.y = y
    out.t = t
    return out

In [ ]:
def test_esdirk_methods():
    
    # Butcher matrix
    A = np.zeros((7,7))
    A[1,0] = 0.26
    A[1,1] = 0.26
    A[2,0] = 0.13
    A[2,1] = 0.84033320996790809
    A[2,2] = 0.26
    A[3,0] = 0.22371961478320505
    A[3,1] = 0.47675532319799699
    A[3,2] = -0.06470895363112615
    A[3,3] = 0.26
    A[4,0] = 0.16648564323248321
    A[4,1] = 0.10450018841591720
    A[4,2] = 0.03631482272098715
    A[4,3] = -0.13090704451073998
    A[4,4] = 0.26
    A[5,0] = 0.13855640231268224
    A[5,2] = -0.04245337201752043
    A[5,3] = 0.02446657898003141
    A[5,4] = 0.61943039072480676
    A[5,5] = 0.26
    A[6,0] = 0.13659751177640291
    A[6,2] = -0.05496908796538376
    A[6,3] = -0.04118626728321046
    A[6,4] = 0.62993304899016403
    A[6,5] = 0.06962479448202728
    A[6,6] = 0.26
    
    Tr = 200.0
    beta = 10.

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    tini = 0.0
    tend = 5.0
    
    # Exemple d'utilisation : Prendre 150 pas de temps avec Beta=10, puis 100, 90, 80 et finalement 70

    nt = 150
    dt = (tend-tini) / (nt-1)

    print(f"  dt = {(tend-tini) / nt}")

    yini = (0.,)
    
    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    if (beta<11. and nt>79): sol_rk4 = rk4(tini, tend, nt, yini, fcn)
    rho = np.max(np.abs(np.linalg.eigvals(np.abs(A))))
    sol_esdirk = DIRK_integration(fun=fcn, y0=np.array(yini), t_span=[tini, tend], nt=nt, jacfun=None, bPrint=False)

    fig_theta = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="\u03B8(t)")
    fig_theta.line(sol_ref.t, sol_ref.y[0], line_width=2, color="crimson", legend_label="Reference")
    if (beta<11. and nt>79): fig_theta.line(sol_rk4.t, sol_rk4.y[0], line_width=2, color="cornflowerblue", legend_label="RK4")
    fig_theta.line(sol_esdirk.t, sol_esdirk.y[0], line_width=2, color="purple", legend_label="ESDIRK5")
    fig_theta.legend.click_policy="hide"
    fig_theta.legend.location = "top_left"
    
    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size): eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_esdirk = np.zeros(sol_esdirk.t.size)
    for it in range(sol_esdirk.t.size): eig_vals_esdirk[it] = jac(0, sol_esdirk.y[:,it])
    if (beta<11. and nt>79):
        eig_vals_rk4 = np.zeros(sol_rk4.t.size)
        for it in range(sol_rk4.t.size): eig_vals_rk4[it] = jac(0, sol_rk4.y[:,it])
            
    fig_eig = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="Valeur propre")
    fig_eig.line(sol_ref.t, eig_vals_ref, color="crimson", line_width=2, legend_label="Reference")
    if (beta<11. and nt>79): fig_eig.line(sol_rk4.t, eig_vals_rk4, color="cornflowerblue", line_width=2, legend_label="RK4")
    fig_eig.line(sol_esdirk.t, eig_vals_esdirk, line_width=2, color="purple", legend_label="ESDIRK5")
    fig_eig.legend.click_policy="hide"
    
    fig_eigdt = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="Valeur propre * dt * rho(A) - implicite / Valeur propre * dt - explicite")
    if (beta<11. and nt>79): fig_eigdt.line(sol_rk4.t, eig_vals_rk4*dt, color="cornflowerblue", line_width=2, legend_label="RK4")
    fig_eigdt.line(sol_esdirk.t, eig_vals_esdirk*dt*rho, line_width=2, color="purple", legend_label="ESDIRK5")
    fig_eigdt.legend.click_policy="hide"

    show(fig_theta)
    show(fig_eig)
    show(fig_eigdt)
    
test_esdirk_methods()

### Radau5 timestep

In [ ]:
def show_scipy_radau_dt():
    
    Tr = 200.0
    beta = 10.

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    tini = 0.0
    tend = 5.0
    nt = 100

    yini = (0.,)
    
    sol_ref   = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-9, rtol=1.e-9)
    sol_radau = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-4, rtol=1.e-4)
    print("Nombre de pas de temps Radau référence 10-9 : ", sol_ref.t.size - 1)
    print("Nombre de pas de temps Radau 10-4 : ",  sol_radau.t.size - 1)
    fig_theta = figure(plot_height=300, plot_width=800, x_range=(tini, tend), title="\u03B8(t)")
    fig_theta.line(sol_ref.t, sol_ref.y[0], line_width=2, color="crimson", legend_label="Reference")
    fig_theta.line(sol_radau.t, sol_radau.y[0], line_width=2, color="blue", legend_label="Tol 10-4")
    fig_theta.legend.location = "top_left"
    fig_theta.legend.click_policy="hide"
    
    fig_dt_radau = figure(plot_height=300, plot_width=800, x_range=(tini, tend), y_range=(1.e-6, 100), y_axis_type="log", title="Pas de temps pour la solution à tolérance 10-4")
    fig_dt_radau.quad(top=sol_radau.t[1:]-sol_radau.t[:-1], bottom=1.e-16, left=sol_radau.t[:-1], right=sol_radau.t[1:],
                fill_color="navy", line_color="white", alpha=0.5) 

    fig_dt = figure(plot_height=300, plot_width=800, x_range=(tini, tend), y_range=(1.e-6, 100), y_axis_type="log", title="Pas de temps pour la solution de référence 10-9")
    fig_dt.quad(top=sol_ref.t[1:]-sol_ref.t[:-1], bottom=1.e-16, left=sol_ref.t[:-1], right=sol_ref.t[1:],
                fill_color="navy", line_color="white", alpha=0.5) 
    


    show(fig_theta)
    show(fig_dt_radau)
    show(fig_dt)
    
show_scipy_radau_dt()